---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
pd.options.display.max_rows = 200
pd.set_option('display.max_columns', 500)

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    file = open('university_towns.txt')
    txt = file.read()
    
    txt = txt.split('\n')[:-1]
    university_towns = []
    for line in txt:
        if '[edit]' in line:
            state = line[:-6]
        else:
            university_towns.append(([state, line.split(' (')[0]]))
    return pd.DataFrame(university_towns, columns=['State', 'RegionName'])


get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df_gdp = pd.read_excel('gdplev.xls', skiprows=220, usecols=[4,6], index_col=0, names=['Quarter', 'GDP'], header=None)
    df_gdp['Recession'] = False
    
    for i in range(2,len(df_gdp) - 2):
        gdp_2ago_q = df_gdp.iloc[i-2,0]
        gdp_last_q = df_gdp.iloc[i-1,0]
        gdp_this_q = df_gdp.iloc[i,0]
        gdp_next_q = df_gdp.iloc[i+1,0]
        gdp_ = df_gdp.iloc[i+2,0]
        last_quarter_is_recession = df_gdp.iloc[i-1,1]
        
        # Start of recession
        if (not last_quarter_is_recession and (gdp_last_q > gdp_this_q > gdp_next_q)):
            df_gdp.iloc[i,1] = True
        # End of recession
        elif (last_quarter_is_recession and (gdp_this_q > gdp_last_q > gdp_2ago_q)):
            df_gdp.iloc[i,1] = False
        # Middle of recession
        else:
            df_gdp.iloc[i,1] = last_quarter_is_recession

    return df_gdp[df_gdp['Recession'] == True].iloc[0].name

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    df_gdp = pd.read_excel('gdplev.xls', skiprows=220, usecols=[4,6], index_col=0, names=['Quarter', 'GDP'], header=None)
    df_gdp['Recession'] = False
    
    for i in range(2, len(df_gdp) - 2):
        gdp_2ago_q = df_gdp.iloc[i-2,0]
        gdp_last_q = df_gdp.iloc[i-1,0]
        gdp_this_q = df_gdp.iloc[i,0]
        gdp_next_q = df_gdp.iloc[i+1,0]
        gdp_ = df_gdp.iloc[i+2,0]
        last_quarter_is_recession = df_gdp.iloc[i-1,1]
        
        # Start of recession
        if (not last_quarter_is_recession and (gdp_last_q > gdp_this_q > gdp_next_q)):
            df_gdp.iloc[i,1] = True
        # End of recession
        elif (last_quarter_is_recession and (gdp_this_q > gdp_last_q > gdp_2ago_q)):
            df_gdp.iloc[i,1] = False
        # Middle of recession
        else:
            df_gdp.iloc[i,1] = last_quarter_is_recession
    for i in range(1, len(df_gdp)):
        if (not df_gdp.iloc[i,1] and df_gdp.iloc[i-1,1]):
            return df_gdp.iloc[i].name
    return

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    
    df_gdp = pd.read_excel('gdplev.xls', skiprows=220, usecols=[4,6], index_col=0, names=['Quarter', 'GDP'], header=None)
    df_gdp['Recession'] = False
    
    for i in range(2, len(df_gdp) - 2):
        gdp_2ago_q = df_gdp.iloc[i-2,0]
        gdp_last_q = df_gdp.iloc[i-1,0]
        gdp_this_q = df_gdp.iloc[i,0]
        gdp_next_q = df_gdp.iloc[i+1,0]
        gdp_ = df_gdp.iloc[i+2,0]
        last_quarter_is_recession = df_gdp.iloc[i-1,1]
        
        # Start of recession
        if (not last_quarter_is_recession and (gdp_last_q > gdp_this_q > gdp_next_q)):
            df_gdp.iloc[i,1] = True
            recession_min = df_gdp.iloc[i,0]
            min_quarter = df_gdp.iloc[i].name
        # End of recession
        elif (last_quarter_is_recession and (gdp_this_q > gdp_last_q > gdp_2ago_q)):
            df_gdp.iloc[i,1] = False
        # No transition
        elif (last_quarter_is_recession):
            df_gdp.iloc[i,1] = last_quarter_is_recession
            if (df_gdp.iloc[i,0] < recession_min):
                recession_min = df_gdp.iloc[i,0]
                min_quarter = df_gdp.iloc[i].name
        else:
            df_gdp.iloc[i,1] = last_quarter_is_recession
            
    return min_quarter


get_recession_bottom()

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df_housing = pd.read_csv('City_Zhvi_AllHomes.csv')
    cols_to_drop = ['Metro', 'CountyName', 'SizeRank', 'RegionID','1996-04', 
                      '1996-05', '1996-06', '1996-07', '1996-08', 
                      '1996-09', '1996-10', '1996-11', '1996-12', 
                      '1997-01', '1997-02', '1997-03', '1997-04', 
                      '1997-05', '1997-06', '1997-07', '1997-08', 
                      '1997-09', '1997-10', '1997-11', '1997-12', 
                      '1998-01', '1998-02', '1998-03', '1998-04', 
                      '1998-05', '1998-06', '1998-07', '1998-08', 
                      '1998-09', '1998-10', '1998-11', '1998-12', 
                      '1999-01', '1999-02', '1999-03', '1999-04', 
                      '1999-05', '1999-06', '1999-07', '1999-08', 
                      '1999-09', '1999-10', '1999-11', '1999-12']
    df_housing = df_housing.drop(cols_to_drop, axis=1)
    df_housing['State'] = df_housing['State'].map(states)
    
#     def to_quarters(row):
#         print(" A whole time", row.name)
#         first_row = False
#         if row.name == 0:
#             first_row = True
#         date_format = '%4d-%02d'
#         quarter_format = '%4dq%d'
#         quarter_price = []
#         for year in range(2000, 2017):
#             for month in range(1, 13):
#                 quarter_price.append(row[date_format % (year, month)])
#                 if month % 3 == 0:
#                     quarter = ((month - 1) // 3) + 1
#                     quarter = quarter_format % (year, quarter)
#                     if first_row:
#                         df_housing[quarter] = False

#                     df_housing.loc[row.name, quarter] = np.mean(quarter_price)
#     df_housing.apply(to_quarters, axis=1)
    
#     date_format = '%4d-%02d'
#     quarter_format = '%4dq%d'
#     for index in df_housing.index:
#         quarter_price = []
#         print(index)
#         for year in range(2000, 2017):
#             for month in range(1, 13):
#                 quarter_price.append(row[date_format % (year, month)])
                
#                 # If end of quarter
#                 if month % 3 == 0:                
#                     quarter = ((month - 1) // 3) + 1
#                     quarter = quarter_format % (year, quarter)
#                     df_housing[quarter] = np.mean(quarter_price)


#     def to_quarter(row, year, quarter):
#         start_quarter = ((i - 1) * 3) + 1
#         last_quarter =  ((i -1) * 3) + 3
#         date_format = '%4d-%02d'
#         quarter_format = '%4dq%d'
#         prices = []
#         for month in range(start_quarter, last_quarter + 1):
#             prices.append(row[date_format % (year, month)])
#         print('reached return', np.mean(prices), "for:", quarter_format % (year, quarter), 'in:', row['Region'])
#         return np.mean(prices)

#     quarter_format = '%4dq%d'
#     date_format = '%4d-%02d'
#     for year in range(2000, 2017):
#         for quarter in range(1,5):
#             df_housing[quarter_format % (year, quarter)] = df_housing.apply(to_quarter, axis=1, args=(year,  quarter))

    return df_housing

convert_housing_data_to_quarters()

,RegionName,State,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,2000-11,2000-12,2001-01,2001-02,2001-03,2001-04,2001-05,2001-06,2001-07,2001-08,2001-09,2001-10,2001-11,2001-12,2002-01,2002-02,2002-03,2002-04,2002-05,2002-06,2002-07,2002-08,2002-09,2002-10,2002-11,2002-12,2003-01,2003-02,2003-03,2003-04,2003-05,2003-06,2003-07,2003-08,2003-09,2003-10,2003-11,2003-12,2004-01,2004-02,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,2005-01,2005-02,2005-03,2005-04,2005-05,2005-06,2005-07,2005-08,2005-09,2005-10,2005-11,2005-12,2006-01,2006-02,2006-03,2006-04,2006-05,2006-06,2006-07,2006-08,2006-09,2006-10,2006-11,2006-12,2007-01,2007-02,2007-03,2007-04,2007-05,2007-06,2007-07,2007-08,2007-09,2007-10,2007-11,2007-12,2008-01,2008-02,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,432600.0,438700.0,440500.0,433900.0,422000.0,415700.0,421200.0,431100.0,435100.0,431900.0,428400.0,430700.0,438800.0,446800.0,455400.0,465500.0,472600.0,478200.0,487600.0,498600.0,508800.0,515300.0,517000.0,517800.0,520800.0,521500.0,523000.0,526300.0,524800.0,519100.0,516200.0,516400.0,516300.0,515500.0,512200.0,509200.0,509800.0,511600.0,512700.0,514000.0,513400.0,510700.0,508100.0,506700.0,505200.0,503700.0,502900.0,502400.0,500500.0,496400.0,491900.0,487500.0,484400.0,481700.0,477900.0,473600.0,469700.0,466100.0,461700.0,457700.0,455300.0,454800.0,456000.0,457800.0,461300.0,466100.0,470200.0,472800.0,475300.0,477100.0,478400.0,479100.0,478900.0,477700.0,476700.0,477100.0,478000.0,478000.0,476800.0,475300.0,473800.0,472000.0,470600.0,469900.0,469500.0,468200.0,465800.0,463500.0,461800.0,460100.0,459700.0,460800.0,461700.0,462500.0,463900.0,466000.0,467500.0,468200.0,468700.0,469400.0,469400,469100.0,468700,469300,470300,472100,474300,477600,481400,485100,488800,492600,495900,499500,503500,506400,509900,515700,520800,522200,522400,523800,526200,528400,529600,530800,532200,533800,536200,540600,545600,551400,557200,563000,568700,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,Los Angeles,California,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,226100.0,228100.0,230600.0,233000.0,235400.0,237300.0,239100.0,240900.0,242900.0,245000.0,247300.0,250100.0,253100.0,255900.0,258800.0,261900.0,265200.0,268600.0,272600.0,276900.0,281800.0,287000.0,292200.0,297000.0,302100.0,307600.0,313400.0,319000.0,324300.0,329600.0,334600.0,339300.0,344500.0,350600.0,356800.0,363400.0,370700.0,378400.0,386500.0,394900.0,404300.0,414600.0,425500.0,436600.0,447400.0,456700.0,464400.0,471200.0,477400.0,483500.0,489100.0,494700.0,501400.0,509700.0,518300.0,527200.0,536100.0,545400.0,555200.0,564500.0,571900.0,576800.0,579700.0,581800.0,583800.0,585300.0,587300.0,589900.0,592200.0,593300.0,593400.0,593100.0,592900.0,591600.0,590900.0,591800.0,592600.0,592100.0,590200.0,586200.0,581600.0,577500.0,572800.0,567600.0,562100.0,554400.0,545000.0,535500.0,525400.0,513600.0,

In [19]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df_housing = pd.read_csv('City_Zhvi_AllHomes.csv')
    cols_to_drop = ['Metro', 'CountyName', 'SizeRank', 'RegionID','1996-04', 
                      '1996-05', '1996-06', '1996-07', '1996-08', 
                      '1996-09', '1996-10', '1996-11', '1996-12', 
                      '1997-01', '1997-02', '1997-03', '1997-04', 
                      '1997-05', '1997-06', '1997-07', '1997-08', 
                      '1997-09', '1997-10', '1997-11', '1997-12', 
                      '1998-01', '1998-02', '1998-03', '1998-04', 
                      '1998-05', '1998-06', '1998-07', '1998-08', 
                      '1998-09', '1998-10', '1998-11', '1998-12', 
                      '1999-01', '1999-02', '1999-03', '1999-04', 
                      '1999-05', '1999-06', '1999-07', '1999-08', 
                      '1999-09', '1999-10', '1999-11', '1999-12']
    df_housing = df_housing.drop(cols_to_drop, axis=1)
    df_housing = df_housing.set_index(['State', 'RegionName'])

    def to_quarter(row, *args):
        prices = []
        for month in args:
            prices.append(row[month])
        return np.mean(prices)

    quarter_format = '%4dq%d'
    date_format = '%4d-%02d'
    for year in range(2000, 2017):
        for quarter in range(1,5):
            if (quarter > 3 and year >= 2016):
                pass
            else:
                opening_month = ((quarter - 1) * 3) + 1
                months = []
                for month in range(opening_month, opening_month + 3):
                    if (year == 2016 and month >= 9):
                        pass
                    else:
                        months.append(date_format % (year, month))
                quarter_name = quarter_format % (year, quarter)
                df_housing[quarter_name] = df_housing.apply(to_quarter, axis=1, args=months)
                df_housing.drop(months, inplace=True, axis=1)
    return df_housing


quarters = convert_housing_data_to_quarters()

In [621]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    


